In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from scipy import stats,sparse
from sklearn.base import TransformerMixin
from datetime import datetime as dt
from math import isnan
from numpy import ma
import pickle as pickle
import xgboost as xgb
import time
from pandas import *
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier 

import json
from sklearn.metrics import roc_curve, auc
from re import sub
from collections import defaultdict
import scipy as sp

from sklearn.cross_validation import StratifiedKFold,cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import linear_model
from sklearn import metrics

from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from scipy import sparse
from sklearn.preprocessing import OneHotEncoder,StandardScaler
#from ModelClassifier import PAClassifier,SGDSVMClassifier,XGBoostClassifier

In [21]:
reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [23]:
test=np.load('Test_NumericalVariable_ConvertedCatVariables.p')
train=np.load('Train_NumericalVariable_ConvertedCatVariables.p')
y=pickle.load(open("./mostrecentinput/ytrain2.dat","rb"))

In [24]:
for seg in [1,2,3,4,5]:
    if seg==1:
        traintest1=np.vstack((train[:,:244],test[:,:244]))
        traintest1=pd.DataFrame(traintest1)
    elif seg==2:
        traintest1=np.vstack((train[:,244:244*2],test[:,244:244*2]))
        traintest1=pd.DataFrame(traintest1)
    elif seg==3:
        traintest1=np.vstack((train[:,244*2:244*3],test[:,244*2:244*3]))
        traintest1=pd.DataFrame(traintest1)
    elif seg==4:
        traintest1=np.vstack((train[:,244*3:244*4],test[:,244*3:244*4]))
        traintest1=pd.DataFrame(traintest1)
    elif seg==5:
        traintest1=np.vstack((train[:,244*4:],test[:,244*4:]))
        traintest1=pd.DataFrame(traintest1)
    else:
        pass



    traintest1=traintest1.apply(lambda x: x - x.min() if x.min() <= 0 else x)
    hot = OneHotEncoder()
    data_mt = hot.fit_transform(traintest1)
    data_mt_train=data_mt[:145231,:]
    data_mt_test=data_mt[145231:,:]
    del data_mt

    Kfeature=round(data_mt_train.shape[1]*0.2)
    SLTKBest=SelectKBest(chi2, k=Kfeature)
    SLTKBest.fit(data_mt_train, y)
    SparseSelect=SLTKBest.get_support()

    if seg==1:
        Xtrain_sparse40Percent_Seg1 = data_mt_train[:,SparseSelect]
        Xtest_sparse40Percent_Seg1 = data_mt_test[:,SparseSelect]
    elif seg==2:
        Xtrain_sparse40Percent_Seg2 = data_mt_train[:,SparseSelect]
        Xtest_sparse40Percent_Seg2 = data_mt_test[:,SparseSelect]
    elif seg==3:
        Xtrain_sparse40Percent_Seg3 = data_mt_train[:,SparseSelect]
        Xtest_sparse40Percent_Seg3 = data_mt_test[:,SparseSelect]
    elif seg==4:
        Xtrain_sparse40Percent_Seg4 = data_mt_train[:,SparseSelect]
        Xtest_sparse40Percent_Seg4 = data_mt_test[:,SparseSelect]
    elif seg==5:
        Xtrain_sparse40Percent_Seg5 = data_mt_train[:,SparseSelect]
        Xtest_sparse40Percent_Seg5 = data_mt_test[:,SparseSelect]
    else:
        pass

del data_mt_train,data_mt_test
del train,test

In [25]:
train40percent=sparse.csr_matrix(sparse.hstack([sparse.coo_matrix(Xtrain_sparse40Percent_Seg1),
                                   sparse.coo_matrix(Xtrain_sparse40Percent_Seg2),             
                                   sparse.coo_matrix(Xtrain_sparse40Percent_Seg3),
                                   sparse.coo_matrix(Xtrain_sparse40Percent_Seg4),
                                   sparse.coo_matrix(Xtrain_sparse40Percent_Seg5),]))

del Xtrain_sparse40Percent_Seg1
del Xtrain_sparse40Percent_Seg2
del Xtrain_sparse40Percent_Seg3
del Xtrain_sparse40Percent_Seg4
del Xtrain_sparse40Percent_Seg5


In [26]:
Kfeature=round(train40percent.shape[1]*0.5)
SLTKBest=SelectKBest(chi2, k=Kfeature)
SLTKBest.fit(train40percent, y)
SparseSelect=SLTKBest.get_support()

In [27]:
train10percent=train40percent[:,SparseSelect]
del train10percent

In [9]:
reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [3]:
FeaturesSelected=np.load('./mostrecentinput/SDGLogistic_Oct172015032138_AUC_0.768923541152.p')

X=np.load('./mostrecentinput/nxtrain_standard_original0.npy')
X1= np.load('./mostrecentinput/nxtrain_standard_derived0.npy')
X2=pd.read_pickle("./mostrecentinput/time_series_derived_standard_train2.dat")
X3=pd.read_pickle("./mostrecentinput/time_series_original_standard_train2.dat")
#X4=pd.read_pickle("./mostrecentinput/cat_numeric_th60_standard_train2.dat")
#X5=pd.read_pickle("./mostrecentinput/cat_le_train2.dat") 
X=np.hstack((X,X1,X2,X3))

y=pickle.load(open("./mostrecentinput/ytrain2.dat","rb"))
xtrain0=X[:,FeaturesSelected]
del X,X1,X2,X3

In [4]:
train10percent=pickle.load(open('train10percent.p','rb'))

In [5]:
xtrain=sparse.csr_matrix(sparse.hstack([sparse.coo_matrix(xtrain0),
                                   sparse.coo_matrix(train10percent)]))
del xtrain0,train10percent

In [6]:
test10percent=pickle.load(open('test10percent.p','rb'))

In [7]:
X_test=np.load('./mostrecentinput/nxtest_standard_original0.npy')
X_test1= np.load('./mostrecentinput/nxtest_standard_derived0.npy')
X_test2=pd.read_pickle("./mostrecentinput/time_series_derived_standard_test2.dat")
X_test3=pd.read_pickle("./mostrecentinput/time_series_original_standard_test2.dat")
#X_test4=pd.read_pickle("./mostrecentinput/cat_numeric_th60_standard_test2.dat")
#X_test5=pd.read_pickle("./mostrecentinput/cat_le_test2.dat") 
X_test=np.hstack((X_test,X_test1,X_test2,X_test3))

xtest0=X_test[:,FeaturesSelected]

In [9]:
del X_test,X_test1,X_test2,X_test3

In [10]:
xtest=sparse.csr_matrix(sparse.hstack([sparse.coo_matrix(xtest0),
                                    sparse.coo_matrix(test10percent)]))

In [ ]:
''' 
Xtrain_sparseOrig=pd.read_pickle("./mostrecentinput/cat_sparse_th60_train2.dat")

SLTKBest=SelectKBest(chi2, k=100)
SLTKBest.fit(Xtrain_sparseOrig, y)
SparseSelect=SLTKBest.get_support()
Xtrain_sparse100 = Xtrain_sparseOrig[:,SparseSelect]

xtrain=sparse.csr_matrix(sparse.hstack([sparse.coo_matrix(xtrain0),
                                   sparse.coo_matrix(Xtrain_sparse100),
                                   sparse.coo_matrix(train10percent)]))

del xtrain0,Xtrain_sparse100,train10percent

X_test=np.load('./mostrecentinput/nxtest_standard_original0.npy')
X_test1= np.load('./mostrecentinput/nxtest_standard_derived0.npy')
X_test2=pd.read_pickle("./mostrecentinput/time_series_derived_standard_test2.dat")
X_test3=pd.read_pickle("./mostrecentinput/time_series_original_standard_test2.dat")
#X_test4=pd.read_pickle("./mostrecentinput/cat_numeric_th60_standard_test2.dat")
#X_test5=pd.read_pickle("./mostrecentinput/cat_le_test2.dat") 
X_test=np.hstack((X_test,X_test1,X_test2,X_test3))

xtest0=X_test[:,FeaturesSelected]
del X_test1,X_test2,X_test3

Xtest_sparseOrig=pd.read_pickle("./mostrecentinput/cat_sparse_th60_test2.dat")

Xest_sparse100 = Xtest_sparseOrig[:,SparseSelect]

xtest=sparse.csr_matrix(sparse.hstack([sparse.coo_matrix(xtest0),
                                   sparse.coo_matrix(Xest_sparse100),
                                    sparse.coo_matrix(test10percent)]))
del xtest0,Xest_sparse100,test10percent

TestID=pd.read_pickle("./mostrecentinput/TestID.p")
'''

In [15]:
n_iterations=500
KFoldNumber=3
kfold = StratifiedKFold(y, KFoldNumber,shuffle=True, random_state=42)
ypreds_test_wl=pd.DataFrame()
ypreds_train_wl=pd.DataFrame()
ypreds_cvindex_wl=pd.DataFrame()
AUCValues=np.zeros(KFoldNumber)

#model = linear_model.SGDClassifier()
clf = linear_model.SGDClassifier(loss='log',average=True,n_iter=n_iterations)
"""
clf = linear_model.SGDClassifier(alpha=0.0031622776601683794, average=True, class_weight='auto',
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0,
       learning_rate='optimal', loss='log', n_iter=500, n_jobs=1,
       penalty='elasticnet', power_t=0.5, random_state=123, shuffle=True,
       verbose=0, warm_start=False)

"""
kfoldnum=0

train_predicted=np.zeros(y.shape)

for train,cv in kfold:
    kfoldnum=kfoldnum+1
    cv_index=np.zeros(y.shape)
   
    MODEL = clf.fit(xtrain[train,:], y[train])
    preds_cv = MODEL.predict_proba(xtrain[cv,:])[:,1]
    preds_fulltrain = MODEL.predict_proba(xtrain)[:,1]
    preds_fulltest = MODEL.predict_proba(xtest)[:,1]
    #preds_cv=preds_fulltrain[cv]
    cv_index[cv]=1
    AUC1=metrics.roc_auc_score(y[cv], preds_cv)
    ypreds_test_wl['CV'+ str(kfoldnum)] = preds_fulltest
    ypreds_train_wl['CV'+ str(kfoldnum)] = preds_fulltrain
    ypreds_cvindex_wl['CV'+ str(kfoldnum)] = cv_index
    AUCValues[kfoldnum-1]=float(AUC1)
    print(AUCValues)

trainPredict=(ypreds_train_wl*ypreds_cvindex_wl).sum(axis=1)
testPredictMean=ypreds_test_wl.mean(axis=1)
AUC1=metrics.roc_auc_score(y, trainPredict)
print(AUC1)

str1=time.strftime("%b%d%Y%H%M%S", time.localtime())
filename0='FinalModelPrediction_Logistic_YF'+str1+'_niter' + str(n_iterations)+ ' _AUC_0p'+ str(int(AUC1.mean()*1e6))
filename=filename0 +'.p'

pickle.dump((ypreds_train_wl,ypreds_test_wl,ypreds_cvindex_wl,AUCValues), 
            open(filename,'wb'), protocol =2) 

save_train_name=filename0 +'_wl_ypredtrain.pkl'
save_test_name=filename0 +'_wl_ypredtest.pkl'

pickle.dump(trainPredict,open(save_train_name,'wb'),protocol=2)
pickle.dump(testPredictMean,open(save_test_name,'wb'),protocol=2)

print('Model prediction done!')

[ 0.78137816  0.          0.        ]
[ 0.78137816  0.78595064  0.        ]
[ 0.78137816  0.78595064  0.78636347]
0.784555656281
Model prediction done!


In [1]:
reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
